# Importing InKind from FileMaker

We use an XML export of the ``inkind`` table.

In [1]:
import os,sys,re,collections
from copy import deepcopy
from functools import reduce
from lxml import etree

First the config.

In [2]:
# Locations

HOME_DIR = os.path.expanduser('~').replace('\\', '/')
BASE_DIR = '{}/projects/has/dacs'.format(HOME_DIR)
TEMP_DIR = '{}/tmp'.format(BASE_DIR)
RESULT_DIR = '{}/sql'.format(BASE_DIR)
INFILE = 'dariah.xml'
INPATH = '{}/{}'.format(BASE_DIR, INFILE)
FMNS = '{http://www.filemaker.com/fmpxmlresult}'
ROW_RAW_FILE = '{}/row_raw_file'.format(TEMP_DIR)
ROW_FILE = '{}/row_file'.format(TEMP_DIR)
ROW_EXT = 'txt'

# data type details

MIN_M = 5       # minimum varchar size = 2**MIN_M
MAX_M = 13      # maximum varchar size = 2**MAX_M
LIMIT_ROWS = 50 # maximum number of rows to be written in one sql insert statement
TYPES = {'number', 'text', 'valuta', 'datetime'}
DATE_PATTERN = re.compile(
    '^\s*([0-9]{2})/([0-9]{2})/([0-9]{4})\s+([0-9]{2}):([0-9]{2})(?::([0-9]{2}))?$'
)

# field management

splitters = dict(
    generic=re.compile('[ \t]*[\n+][ \t\n]*'),
    generic_comma=re.compile('[ \t]*[\n+,][ \t\n]*'),
)
SPLIT_FIELDS = dict(
    disciplines_associated=splitters['generic'],
    other_keywords=splitters['generic_comma'],
    tadirah_research_activities=splitters['generic'],
    tadirah_research_objects=splitters['generic'],
    tadirah_research_techniques=splitters['generic'],
    type_of_inkind=splitters['generic'],
    vcc=splitters['generic'],
)

SKIP_FIELDS = {
    'teller',
    'whois',
}

FIELD_TYPE_OVERRIDE = dict(
    costs_total='valuta',
    total_costs_total='valuta',
    whois='text',
    creation_date_time='datetime',
    modification_date_time='datetime',
    dateandtime_approval='datetime',
    dateandtime_cioapproval='datetime',
    dateandtime_ciozero='datetime',
)

MERGE_FIELDS = dict(
    academic_entity_url=('academic_entity_url_2',),
    contribution_url=('contribution_url_2',),
    contact_person_mail=('contact_person_mail_2',),
    gnewpassword=('gnewpassword2',),
    vcc_head_decision_vcc1=('vcc_head_decision_vcc11', 'vcc_head_decision_vcc12'),
    vcc_head_decision_vcc2=('vcc_head_decision_vcc21', 'vcc_head_decision_vcc22'),
    vcc_head_decision_vcc3=('vcc_head_decision_vcc31', 'vcc_head_decision_vcc32'),
    vcc_head_decision_vcc4=('vcc_head_decision_vcc41', 'vcc_head_decision_vcc42'),
    vcc_name=(
        'vcc11_name', 'vcc12_name', 
        'vcc21_name', 'vcc22_name',
        'vcc31_name', 'vcc32_name',
        'vcc41_name', 'vcc42_name',
    ),
)

VALUE_FIELDS = {
    'country',
}

NULL_VALUES = {
    'http://',
    'https://',
}

# table organization

MAIN_TABLE = 'contrib'

UNRELATED_TABLES = {
    'help': {'help_text', 'help_description'},
}

DB_NAME = 'inkind_data'

merge_errors = 0
merge_fields = {}
for (mfhead, mftail) in MERGE_FIELDS.items():
    for f in mftail:
        if f in merge_fields:
            print(
                'WARNING: field `{}` already merged into `{}` now to be merged into `{}`'.format(
                    f, merge_field[f], mfhead,
            ))
            merge_errors += 1
        merge_fields[f] = mfhead

if merge_errors:
    print('There were {} merge errors'.format(merge_errors))
else:
    print('merge OK')

good = True
for x in [1]:
    good = False
    if not os.path.exists(BASE_DIR):
        print('BASE_DIR does not exist: {}'.format(BASE_DIR))
        break
    this_good = True
    for cdir in (TEMP_DIR, RESULT_DIR):
        this_good = False
        if not os.path.exists(cdir):
            try:
                os.makedirs(cdir)
            except os.error as e:
                print('{} could not be created.'.format(cdir))
                break
        this_good = True
    if not this_good:
        break
    good = True
if not good:
    print('There were configuration errors')
else:
    print('Configuration OK')

merge OK
Configuration OK


## Parse the XML file

In [3]:
parser = etree.XMLParser(remove_blank_text=True, ns_clean=True)
root = etree.parse(INPATH, parser).getroot()

## Get the fields and their types

In [4]:
fieldroots = [x for x in root.iter(FMNS+'METADATA')]
fieldroot = fieldroots[0]
tfields = []
field_defs = {}
for x in fieldroot.iter(FMNS+'FIELD'):
    fname = x.get('NAME').lower().replace(' ','_')
    ftype = FIELD_TYPE_OVERRIDE.get(fname, None) or x.get('TYPE').lower()
    fmult = int(x.get('MAXREPEAT'))
    if fname in SPLIT_FIELDS: fmult += 1
    tfields.append(fname)
    field_defs[fname] = [ftype, fmult]
    if ftype not in TYPES:
        print('WARNING: field `{}` has unknown type "{}"'.format(fname, ftype))

merge_errors = 0
for f in merge_fields:
    if f not in field_defs:
        print(
            'WARNING: Cannot merge unknown field `{}`'.format(
            f,
        ))
        merge_errors += 1
        continue
    ftarget = merge_fields[f]
    (ftype, fmult) = field_defs[f]
    if ftarget not in field_defs:
        field_defs[ftarget] = [ftype, 0]
    (ttype, tmult) = field_defs[ftarget]
    if ttype != ftype:
        print(
            'WARNING: field `{}` of type "{}" is merged into field `{}` of other type "{}"'.format(
                f, ftype, ftarget, ttype,
        ))
        merge_errors += 1
    field_defs[ftarget][1] += fmult
    del field_defs[f]

for f in SKIP_FIELDS: del field_defs[f]

fields = sorted({f for f in tfields if f in field_defs} | set(merge_fields.values()))

if merge_errors:
    print('There were {} merge errors'.format(merge_errors))
else:
    print('MERGE OK')

print('{} fields, some are skipped or merged\n{} retained fields:\n{}\n'.format(
    len(tfields), len(fields), 
    '\n'.join('{:<8}{} {:<20}'.format(
        *field_defs[f], 
        f, 
    ) for f in fields)
))

MERGE OK
60 fields, some are skipped or merged
43 retained fields:
text    3 academic_entity_url 
number  1 approved            
text    4 contact_person_mail 
text    1 contact_person_name 
text    3 contribution_url    
text    1 costs_description   
valuta  1 costs_total         
text    1 country             
datetime1 creation_date_time  
text    1 creator             
datetime8 dateandtime_approval
datetime1 dateandtime_cioapproval
datetime1 dateandtime_ciozero 
text    1 description_of_contribution
text    2 disciplines_associated
text    2 gnewpassword        
text    1 goldpassword        
text    1 help_description    
text    1 help_text           
number  1 ikid                
number  1 ikid_base           
text    1 last_modifier       
text    1 message             
datetime1 modification_date_time
text    2 other_keywords      
text    1 other_type_of_inkind
number  1 submit              
text    2 tadirah_research_activities
text    2 tadirah_research_objects
text    2

## Get the data

In [5]:
dataroots = [x for x in root.iter(FMNS+'RESULTSET')]
dataroot = dataroots[0]
rows_raw = []
errors = collections.defaultdict(list)
    
for (i, r) in enumerate(dataroot.iter(FMNS+'ROW')):
    row = []
    for c in r.iter(FMNS+'COL'):
        data = [x.text for x in c.iter(FMNS+'DATA')]
        row.append(data)
    if len(row) != len(tfields):
        errors['Number of fields'].append(i)
    rows_raw.append(row)
if errors:
    for k in sorted(errors):
        print('{:<20}: {}'.format(k, ','.join(errors[k])))
else:
    print('{:>5} rows read'.format(len(rows_raw)))

rf = open('{}.{}'.format(ROW_RAW_FILE, ROW_EXT), 'w')
for row in rows_raw:
    for (fname, values) in zip(tfields, row):
        rf.write('@{:>30} = {}\n'.format(
            fname,
            ' | '.join('{}'.format(v) for v in values),
        ))
    rf.write('{}\n'.format('='*100))
rf.close()

  309 rows read


## Organize the tables

In [6]:
unrelated_fields = {}
good = True
for ut in UNRELATED_TABLES:
    for uf in UNRELATED_TABLES[ut]:
        if uf not in field_defs:
            print(
                'WARNING: unrelated table `{}`: unknown field `{}`'.format(
                    ut, uf,
            ))
            good = False
        unrelated_fields[uf] = ut
if good:
    print('Table organization OK')
else:
    print('Errors in table organization')

def freeze(row): return tuple((x,tuple(sorted(y))) for (x,y) in sorted(row.items()))
def unfreeze(row): return dict((x, set(y)) for (x,y) in row)

rows_part = collections.defaultdict(list)

for row in rows_raw:
    row_part = collections.defaultdict(dict)
    for (fname, values) in zip(tfields, row):
        target_table = unrelated_fields.get(fname, MAIN_TABLE)
        row_part[target_table][fname] = values
    for t in row_part:
        rows_part[t].append(row_part[t])

for t in UNRELATED_TABLES:
    new_rows = [unfreeze(frow) for frow in {freeze(row) for row in rows_part[t]}]
    rows_part[t] = new_rows

for t in rows_part:
    rf = open('{}_{}.{}'.format(ROW_RAW_FILE, t, ROW_EXT), 'w')
    print('Table `{}`: {:>5} rows'.format(t, len(rows_part[t])))
    for row in rows_part[t]:
        for (fname, values) in sorted(row.items()):
            rf.write('@{:>30} = {}\n'.format(
                fname,
                ' | '.join('{}'.format(v) for v in values),
            ))
        rf.write('{}\n'.format('='*100))
    rf.close()

Table organization OK
Table `contrib`:   309 rows
Table `help`:     2 rows


## Transform the values

Various non-informational values will be converted to NULL.
Values will be thinned: 
Identical values will be reduced to one copy.

In [19]:
def date_repl(match):
    [d,m,y,hr,mn,sc] = list(match.groups())
    return '{}-{}-{}T{}:{}:{}'.format(y,m,d,hr,mn,sc or '00')
    
def sq(v_raw):
    return "'{}'".format(
        v_raw.strip().replace("'","''").replace('\t', '\\t').replace('\n', '\\n')
    )

def num(v_raw, i, fname):
    if v_raw.isdigit(): return int(v_raw)
    print(
        'WARNING: field `{}` record {}: not an integer: "{}"'.format(
            fname, i, v_raw
    ))
    return v_raw

def money(v_raw, i, fname):
    if '€' not in v_raw:
        print(
            'WARNING: field `{}` record {}: no currency symbol: "{}"'.format(
                fname, i, v_raw
        ))
    if ',' in v_raw:
        print(
            'WARNING: field `{}` record {}: comma in amount: "{}"'.format(
                fname, i, v_raw
        ))
    return num(v_raw.replace('€', '').replace('.','').replace(',', ''), i, fname)

def dtm(v_raw, i, fname):
    if not DATE_PATTERN.match(v_raw):
        print(
            'WARNING: field `{}` record {}: not a valid date time: "{}"'.format(
                fname, i, v_raw
        ))
        return v_raw
    return("'{}'".format(DATE_PATTERN.sub(date_repl, v_raw)))

def transform_rows(t):
    if t not in rows_part:
        print('WARNING: Unknown table `{}`'.format(t))
    rows = []
    for (i, row_raw) in enumerate(rows_part.get(t, [])):
        values = {}
        for (fname, values_raw) in sorted(row_raw.items()):
            if fname in SKIP_FIELDS: continue
            sep = SPLIT_FIELDS.get(fname, None)
            if sep != None:
                values_raw = sorted(reduce(
                    set.union, 
                    [set(sep.split(v)) for v in values_raw if v != None], 
                    set(),
                ))
                if '' in values_raw: values_raw.remove('')
            ftarget = merge_fields.get(fname, fname)
            (ftype, fmult) = field_defs[ftarget]
            valset = set()
            for v_raw in values_raw:
                if v_raw == None or v_raw in NULL_VALUES: v = 'NULL'
                elif ftype == 'text': v = sq(v_raw)
                elif ftype == 'number': v = num(v_raw, i, fname)
                elif ftype == 'valuta': v = money(v_raw, i, fname)
                elif ftype == 'datetime': v = dtm(v_raw, i, fname)
                else: v = v_raw
                valset.add(v)
            if fmult > 1: valset.discard('NULL')
            these_values = values.setdefault(ftarget, set())
            these_values |= valset
        rows.append(values)
    print('Table `{}`: {:>5} rows checked'.format(t, len(rows)))

    rf = open('{}_{}.{}'.format(ROW_FILE, t, ROW_EXT), 'w')
    for row in rows:
        for (fname, values) in sorted(row.items()):
            rf.write('@{:>30} = {}\n'.format(
                fname,
                ' | '.join('{}'.format(v) for v in sorted(values)),
            ))
        rf.write('{}\n'.format('='*100))
    rf.close()
    return rows

rows = {}
for t in rows_part: rows[t] = transform_rows(t)

Table `contrib`:   309 rows checked
Table `help`:     2 rows checked


## Turn the data into a dict

We represent the data with a dictionary. The keys are the field names.
The values are dictionaries again, with keys new ids and with values the value that the row with that id has for that field.

In [20]:
field_data = {}

def pprintf(tname, fname):
    values_raw = field_data[tname][fname]
    values = sorted(v for v in reduce(set.union, values_raw, set()) if v != 'NULL')
    print('\n'.join('{}'.format(v) for v in values))
    
for t in rows:
    for row in rows[t]:
        for (fname, values) in sorted(row.items()):
            field_data.setdefault(t, {}).setdefault(fname, []).append(values)
    print('Table `{}`: {:<5} records and {:<2} fields compiled'.format(
        t, len(rows[t]), len(field_data[t]),
    ))

Table `contrib`: 309   records and 41 fields compiled
Table `help`: 2     records and 2  fields compiled


In [21]:
# check
good = True
for t in field_data:
    for f in field_data[t]:
        if len(field_data[t][f]) != len(rows[t]):        
            print(
                'WARNING: table `{}`, field `{}`: wrong number of records: {} instead of {}'.format(
                    t, f, len(field_data[t][f]), len(rows[t]),
            ))
            good = False
if good:
    print('OK')
else:
    print('There were errors')

OK


In [22]:
pprintf(MAIN_TABLE, 'vcc_name')

'Dirk Wintergrün'
'Hansmichael Hohenegger'
'Hella Hollander'
'Marianne Huan'
'Sophie David'
'Susan Schreibman'
'Tibor Kálmán'


## Extract related and related tables

In [23]:
def getsize(source, fname):
    values = set()
    for vals in source: values |= set(vals)
    maxlen = max({len(x) for x in values if x != 'NULL'}, default=0)
    result = 0
    for m in range(MIN_M, MAX_M+1):
        if maxlen <= 2**m:
            result = m
            break
    if maxlen > 2**MAX_M:
        print(
            'Field `{}`: value with length {} gets type TEXT'.format(
                fname, maxlen, 2**MAX_M,
        ))
        return False
    return 2**m

def getdef(source, fname, newfname, warn_mult=True):
    (ft, fmult) = field_defs[fname]
    if warn_mult and fmult > 1:
        print(
            'WARNING: skipping field `{}` because it contains multiple values'.format(
                fname,
        ))
        return None
    if ft == 'number':
        ftype = 'int'
        fsize = '(4)'
        fext = ''
    elif ft == 'text':
        ftype = 'varchar'
        fsize_raw = getsize(source, fname)
        if not fsize_raw:
            ftype = 'text'
            fsize = ''
        else:
            fsize = '({})'.format(fsize_raw)
        fext = 'character set utf8'
    elif ft == 'valuta':
        ftype = 'decimal'
        fsize = '(10,2)'
        fext = ''
    elif ft == 'datetime':
        ftype = 'datetime'
        fsize = ''
        fext = ''
    else:
        print('WARNING: skipping field `{}` because it has unknown type `{}`'.format(
            fname, ft,
        ))
        return None
    return '{} {}{} {}'.format(newfname, ftype, fsize, fext)

def getrdef(fname):
    return '''{fn}_id int(4),
    foreign key ({fn}_id) references {fn}(id)'''.format(fn=fname)

def sql_data(df, tname, flist, rows):
    head = 'insert into {} ({}) values'.format(tname, ','.join(flist))
    for (i, row) in enumerate(rows):
        if i % LIMIT_ROWS == 0:
            if i > 0: df.write(';')
            df.write('\n')
            df.write('select "table {} row {}" as " ";\n'.format(tname, i))
            df.write(head)
            sep = ''
        df.write('\n{}\t'.format(sep))
        sep = ','
        df.write('({})'.format(','.join(str(x) for x in row)))
    df.write(';\n')
        
def print_maintables(maindata, reltables, cf, df):
    for t in maindata:
        fdefs = ['id int(4) primary key']
        flist = sorted(maindata[t])
        fnewlist = []
        for fname in flist:
            if unrelated_fields.get(fname, MAIN_TABLE) != t: continue
            if fname in reltables:
                fdef = getrdef(fname)
                fnewname = '{}_id'.format(fname)
            else:
                fdef = getdef(field_data[t][fname], fname, fname)
                fnewname = fname
            fdefs.append(fdef)
            fnewlist.append(fnewname)
        cf.write('''
create table {} (
    {}
);
    '''.format(t, ',\n\t'.join(fdefs)))
        maintable_raw = zip(*(maindata[t][f] for f in flist))
        maintable = [
            [i]+[sorted(vals)[0] for vals in row] for (i, row) in enumerate(maintable_raw)
        ]
        sql_data(df, t, ['id'] + fnewlist, maintable)

def print_reltables(reltables, relvalues, cf, df):
    for tname in sorted(reltables):
        fdefs = ['id int(4) primary key']
        fdef = getdef([relvalues[tname].keys()], tname, 'val', warn_mult=False)
        if fdef == None: continue            
        fdefs.append(fdef)
        cf.write('''
create table {} (
    {}
);
'''.format(tname, ',\n\t'.join(fdefs)))
        sql_data(df, tname, ['id', 'val'], reltables[tname])

def print_relxtables(relxtables, cf, df):
    for tname in sorted(relxtables):
        tname_rep = '{}_{}'.format(MAIN_TABLE, tname)
        main_id = '{}_id'.format(MAIN_TABLE)
        val_id = '{}_id'.format(tname)
        fdefs = '''
    {mi} int(4),
    {vi} int(4),
    foreign key ({mi}) references {mt}(id),
    foreign key ({vi}) references {tn}(id)
'''.format(mt=MAIN_TABLE, mi=main_id, tn=tname, vi=val_id)
        cf.write('''
create table {} ({});
'''.format(tname_rep, fdefs))
        sql_data(df, tname_rep, [main_id, val_id], relxtables[tname])

def extract(maindata, relvalues, relindex, reltables, relxtables, fname):
    is_single = field_defs[fname][1] == 1 # single value of multiple values
    t = unrelated_fields.get(fname, MAIN_TABLE)
    for (i, values) in enumerate(field_data[t][fname]):
        for value in values:
            vid = relvalues[fname].get(value, None)
            if vid == None:
                relindex[fname] += 1
                vid = relindex[fname]
                reltables[fname].append((vid, value))
            relvalues[fname][value] = vid
            if is_single:
                maindata[t][fname][i] = [vid]
            else:
                relxtables[fname].append((i, vid))
    if not is_single: del maindata[t][fname]

def model_data(field_list):
    maindata = deepcopy(field_data)
    relvalues = collections.defaultdict(dict)
    relindex = collections.Counter()
    reltables = collections.defaultdict(list)
    relxtables = collections.defaultdict(list)

    for fname in field_list:
        if fname not in field_defs:
            print('ERROR: wrong field {}'.format(fname))
            continue
        extract(
            maindata, relvalues, relindex, reltables, relxtables, fname,
        )
    return (maindata, reltables, relxtables, relvalues)

def transform_data():
    mult_fields = {f for f in fields if field_defs[f][1] > 1}
    (maindata, reltables, relxtables, relvalues) = model_data(
        VALUE_FIELDS | mult_fields
    )
    cf = open('{}/create.sql'.format(RESULT_DIR), 'w')
    df = open('{}/data.sql'.format(RESULT_DIR), 'w')
    df.write('''
select "FILL TABLES OF DATABASE {db}" as " ";

use {db};

'''.format(db=DB_NAME))

    cf.write('''
select "CREATE DATABASE {db} AND TABLES" as " ";

drop database if exists {db};
create database {db} character set utf8;
use {db};

'''.format(db=DB_NAME))
    cf.write('/* value tables */\n')
    df.write('\n/* value tables */\n')
    print_reltables(reltables, relvalues, cf, df)
    
    cf.write('/* main tables */\n')
    df.write('\n/* main tables */\n')
    print_maintables(maindata, reltables, cf, df)

    cf.write('/* cross tables */\n')
    df.write('\n/* cross tables */\n')
    print_relxtables(relxtables, cf, df)
    
    cf.close()
    df.close()
    print('SQL written')

transform_data()

Field `description_of_contribution`: value with length 12856 gets type TEXT
SQL written
